## Imports

In [1]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

## Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

## Create Model

Now we use the Bar/QR-code Scanner Model Package to create a model

In [3]:
from src.model_package_arns import ModelPackageArnProvider

In [8]:
from sagemaker import ModelPackage
modelpackage_arn = ModelPackageArnProvider.get_model_package_arn(sess.boto_region_name)

# Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='image/jpeg')

# Create a deployable model
model = ModelPackage(role=role,
                                      model_package_arn = modelpackage_arn,
                                      sagemaker_session = sess,
                                      predictor_cls = predict_wrapper)
# ARN
print(modelpackage_arn)

arn:aws:sagemaker:us-east-2:057799348421:model-package/bar-qr-code-reader-1571413449-a35a47dda86b16474ecd69ff9f20b46f


### Live Inference Endpoint

Now we demonstrate the creation of an endpoint for live inference

In [9]:
# Deploy the model
predictor = model.deploy(1, 'ml.m4.xlarge', endpoint_name='barcode-endpoint')

ClientError: An error occurred (ValidationException) when calling the CreateEndpoint operation: Cannot create already existing endpoint "arn:aws:sagemaker:us-east-2:428712150059:endpoint/barcode-endpoint".

### Testing on a single file 

The model takes an image file (.jpg/.png) as input. We give the location of the sample input file and prepare it for payload. An example has been shown in the following code shell.

We get the response for our request as a dictionary of Barcode Data and the Barcode Type associated with the barcode. In case of a QR code, the Barcode Type is returned as "QR-code".

In [6]:
file_name = 'data/test_files/QRcode_example5.jpg'

#### Preparing the input file for payload and viewing the response

In [7]:
# Open the file and read the image into a bytearray.
with open(file_name, "rb") as image:
  b = bytearray(image.read())

# Perform a prediction
result = predictor.predict(b).decode('utf-8')

# View the prediction
print(result)

{"upi://pay?aid=uGICAgICAg_ziOw&pa=nishanthballal.9-1@okicici&pn=Nishanth%20Ballal": "QRCODE"}


## Batch Transform Job

Now let's use the model built to run a batch inference job and verify it works.

### Batch Transform Input Preparation


In [10]:
bucket = sess.default_bucket()

transform_input_prefix = "barcode/validation"
TRANSFORM_WORKDIR = "data/transform" 
# upload data from local directory to bucket

transform_input = sess.upload_data(TRANSFORM_WORKDIR, key_prefix=transform_input_prefix) 

print ("Transform Data Location " + transform_input)

Transform Data Location s3://sagemaker-us-east-2-428712150059/barcode/validation


### Run Batch Transform

Now that our batch transform input is setup, we run the transformation job next

In [12]:
transformer = model.transformer(1, 'ml.m4.xlarge', 
                                output_path="s3://"+bucket+"/barcode/batch-transform-output")
transformer.transform(transform_input, content_type='image/jpeg')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

Using already existing model: bar-qr-code-reader-1571413449-a35a47dda-2019-10-30-10-11-47-607


..................
Starting the inference server with 4 workers.
[2019-10-30 10:18:26 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2019-10-30 10:18:26 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2019-10-30 10:18:26 +0000] [10] [INFO] Using worker: gevent
[2019-10-30 10:18:26 +0000] [14] [INFO] Booting worker with pid: 14
[2019-10-30 10:18:26 +0000] [15] [INFO] Booting worker with pid: 15
[2019-10-30 10:18:26 +0000] [19] [INFO] Booting worker with pid: 19
[2019-10-30 10:18:26 +0000] [20] [INFO] Booting worker with pid: 20
169.254.255.130 - - [30/Oct/2019:10:18:39 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [30/Oct/2019:10:18:39 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Step 1
Step 2
Step3
Endpoint invoked
169.254.255.130 - - [30/Oct/2019:10:18:39 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [30/Oct/2019:10:18:39 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "G

In [14]:
# Print the outputs

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sess.default_bucket())

for object_summary in my_bucket.objects.filter(Prefix="barcode/batch-transform-output"+ "/"):
   file_name = object_summary.key.split('/')[-1]
   print(file_name)
   s3.Bucket(sess.default_bucket()).download_file("barcode/"+"batch-transform-output" +"/"+ file_name,  'batch_results')
   with open('batch_results') as f:
       results = f.readlines()  
   
   print("".join(results))

QRcode_example5.jpg.out
{"upi://pay?aid=uGICAgICAg_ziOw&pa=nishanthballal.9-1@okicici&pn=Nishanth%20Ballal": "QRCODE"}
barcode_example.png.out
{"{\"author\": \"Adrian\", \"site\": \"PyImageSearch\"}": "QRCODE", "https://www.pyimagesearch.com/": "QRCODE", "PyImageSearch": "QRCODE", "AdrianRosebrock": "CODE128"}
barcode_example3.jpg.out
{"14MF3IM01": "CODE39"}


### Clean Up Endpoint

In [13]:
predictor.delete_endpoint()